# 07.2.自动求导实现
假设想对函数 $y=2x^Tx$ 关于列向量 $x$ 求导

In [1]:
import torch
x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

In [2]:
x.requires_grad_(True)
#等价于 x = torch.arange(4.0,requeires_grad=True)
x.grad #默认值是None

In [3]:
# y = 2 * x * x
y = 2 * torch.dot(x,x) # 2 * (0*0 + 1*1 + 2*2 + 3*3)
y

tensor(28., grad_fn=<MulBackward0>)

In [4]:
y.backward()
x.grad # y' = 4*x

tensor([ 0.,  4.,  8., 12.])

In [5]:
x.grad == 4 * x

tensor([True, True, True, True])

In [6]:
x.grad.zero_() #如果不写这一行，新的梯度会累加到旧的梯度中
y=x.sum()
y.backward()
x.grad # y' = 1

tensor([1., 1., 1., 1.])

In [7]:
x.grad.zero_()
y = x * x #这里*是对应位置元素相乘
y.sum().backward()#等价于y.backward(torch.ones(len(x)))
x.grad #y' = 2*x

tensor([0., 2., 4., 6.])

将某些计算移动到记录的计算图之外

In [8]:
x.grad.zero_()
y = x * x
u = y
u.requires_grad_(True)
z = u * x
z.sum().backward()
'''
z = x^3
z' = 3*x^2
'''
print(x.grad)
# print(u.grad) #这一行会报错，求导结果只会保存在叶节点中，此处变量x是叶节点而u不是
print(u)

tensor([ 0.,  3., 12., 27.])
tensor([0., 1., 4., 9.], grad_fn=<MulBackward0>)


In [9]:
x.grad.zero_()
y = x * x
u = y.detach()
z = u * x
z.sum().backward()
'''
z = u*x
z' = u
'''
print(x.grad)
print(u)

tensor([0., 1., 4., 9.])
tensor([0., 1., 4., 9.])


In [10]:
'''
上面z对x求导时，u被当成一个常数了
但是y对x求导时，y仍然是一个关于x的函数
'''
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])